In [2]:
import pandas as pd
from pathlib import Path

# 상위 폴더 경로
root_dir = Path("../Data/Train_Data/1_RawDataset")
hand = "Right"

# 피험자별, 행동별 누적 시간을 저장할 Dict { participant: { action: cumulative_time } }
cumulative_times = {}

for participant_folder in root_dir.iterdir():
    if participant_folder.is_dir():
        participant_id = participant_folder.name
        cumulative_times.setdefault(participant_id, {})
        
        file_path = participant_folder / hand
        
        # SensorData.csv 파일 찾기
        sensor_files = list(file_path.glob("*SensorData.csv"))
        
        for sensor_file in sensor_files:
            # 확장자 제거
            stem = sensor_file.stem  
            parts = stem.split("_")

            # 행동 찾기
            if len(parts) < 8:
                continue
            action_tokens = parts[1:-7]
            action = "_".join(action_tokens)
            if not action:
                continue

            # CSV 읽기
            try:
                df = pd.read_csv(sensor_file)
            except Exception as e:
                print(f"Error reading {sensor_file}: {e}")
                continue
            
            if df.empty or 'Time' not in df.columns:
                continue
            
            # 마지막 행의 Time 값 (경과시간)
            last_time = df['Time'].iloc[-1]
            
            # 행동별 누적 시간 업데이트 (초 단위)
            cumulative_times[participant_id].setdefault(action, 0)
            cumulative_times[participant_id][action] += last_time

# 원하는 행동 리스트
desired_actions = ["Shower", "Tooth_brushing", "Washing_hands", "Vacuum_Cleaner", "Wiping"]

# 피험자 별 누적 시간을 DataFrame 형태로 변환
# 원하는 행동에 해당하지 않는 항목은 "Other"로 합산
data_rows = []
for participant, actions in cumulative_times.items():
    # 초기화: 원하는 행동은 0, Other도 0
    participant_data = {action: 0 for action in desired_actions}
    participant_data["Other"] = 0

    for action, total_seconds in actions.items():
        minutes = total_seconds / 60.0  # 분 단위 변환
        if action in desired_actions:
            participant_data[action] += minutes
        else:
            participant_data["Other"] += minutes

    participant_data = {k: round(v, 2) for k, v in participant_data.items()}
    participant_data["Participant"] = participant
    data_rows.append(participant_data)

# DataFrame 생성 및 피험자 번호를 행 인덱스로 지정
result_df = pd.DataFrame(data_rows)
result_df.set_index("Participant", inplace=True)
result_df = result_df[desired_actions + ["Other"]]  # 열 순서 지정

print(result_df)

# CSV 파일로 저장
output_csv_path = "../Data/Train_Data/Collected_Data_Time.csv"
result_df.to_csv(output_csv_path, encoding='utf-8-sig')
print(f"CSV 파일 저장 완료: {output_csv_path}")

             Shower  Tooth_brushing  Washing_hands  Vacuum_Cleaner  Wiping  \
Participant                                                                  
10            60.67           62.69          63.36           50.67   60.67   
100           30.00           10.00          10.00           10.00    3.00   
101           25.02           25.02          30.02           40.02   25.01   
102           50.02           40.02          40.03           65.21   60.03   
103           20.01            9.98          10.01           20.01   20.01   
104            0.00            5.00           5.00            5.01    5.00   
3              5.17            6.68           6.68            5.17    5.17   
4              0.00            6.68           6.68            0.00    0.00   

              Other  
Participant          
10           647.29  
100          180.01  
101          145.05  
102          475.07  
103           20.02  
104            5.00  
3             51.90  
4             48.98  